In [1]:
import polars as pl
from test_cfg import *

In [2]:
all_df, daily_df = get_dfs()

In [15]:
daily_pdf = daily_df.to_pandas()

In [3]:
all_df

stock,date,time,midEnd,trade,volume
cat,date,time,f64,i32,i32
"""ADSK""",2019-01-02,09:30:00,125.205,-287,287
"""ADSK""",2019-01-02,09:30:10,124.61,20,20
"""ADSK""",2019-01-02,09:30:20,124.61,0,0
"""ADSK""",2019-01-02,09:30:30,125.155,200,200
"""ADSK""",2019-01-02,09:30:40,125.705,-385,385
…,…,…,…,…,…
"""ADS""",2019-12-27,15:59:20,110.915,0,0
"""ADS""",2019-12-27,15:59:30,110.95,0,0
"""ADS""",2019-12-27,15:59:40,110.92,0,0


In [ ]:
daily_pdf.pivot(index='stock', columns='date', values='adv').T.plot()

In [51]:
daily_df.filter(pl.col('stock') == 'ADS')

stock,date,adv,vol
cat,date,f64,f64
"""ADS""",2019-01-02,null,null
"""ADS""",2019-01-03,null,null
"""ADS""",2019-01-04,null,null
"""ADS""",2019-01-07,null,null
"""ADS""",2019-01-08,null,null
…,…,…,…
"""ADS""",2019-12-19,90423.619048,0.000303
"""ADS""",2019-12-20,88878.952381,0.000304
"""ADS""",2019-12-23,85781.4,0.000297


In [6]:
import sys
sys.path.append('../')
from models.impact_model import *

In [12]:
trade_df = all_df

In [13]:
trade_df = trade_df.join(daily_df, how='left', on=['stock', 'date'])

In [20]:
ow = OWModel(0.98, 100000)
impacted_df = ow.subtract_impact(trade_df).collect()

In [21]:
# Is vol too low? need to multiply by 100?
impacted_df.filter(pl.col('impacted_price').is_not_null())

stock,date,time,midEnd,trade,volume,adv,vol,impact,impacted_price
cat,date,time,f64,i32,i32,f64,f64,f64,f64
"""ADSK""",2019-02-04,09:30:00,149.25,-404,404,237350.7,0.000308,-0.001071,149.248929
"""ADSK""",2019-02-04,09:30:10,149.22,0,0,237350.7,0.000308,-0.00105,149.21895
"""ADSK""",2019-02-04,09:30:20,149.22,0,0,237350.7,0.000308,-0.001029,149.218971
"""ADSK""",2019-02-04,09:30:30,149.22,0,0,237350.7,0.000308,-0.001008,149.218992
"""ADSK""",2019-02-04,09:30:40,149.22,0,0,237350.7,0.000308,-0.000988,149.219012
…,…,…,…,…,…,…,…,…,…
"""ADS""",2019-12-26,15:59:20,110.285,-85,85,81498.9,0.000286,0.002311,110.287311
"""ADS""",2019-12-26,15:59:30,110.27,-88,88,81498.9,0.000286,0.001634,110.271634
"""ADS""",2019-12-26,15:59:40,110.335,-100,100,81498.9,0.000286,0.000885,110.335885
